In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/아이펠/DLthon_pepero_day"

/content/drive/MyDrive/Colab Notebooks/아이펠/DLthon_pepero_day


In [ ]:
from dataset import DKTCDataset, collate_fn, create_dataloaders

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time

# 모델, 함수 정의

In [ ]:
class CNNClassifier(nn.Module):
    """
    1D CNN 기반 텍스트 분류 모델
    """
    def __init__(self,
                 vocab_size,      # 어휘 사전의 크기 (vocab 객체로부터 받음)
                 embed_dim,       # 임베딩 벡터의 차원
                 num_classes,     # 분류할 클래스의 개수 (5)
                 num_filters,     # 각 필터 크기별 컨볼루션 필터의 수
                 filter_sizes,    # 사용할 컨볼루션 필터의 크기
                 dropout_prob):   # 드롭아웃 확률

        super(CNNClassifier, self).__init__()

        # 1. 임베딩 레이어
        # padding_idx=0: <PAD> 토큰은 0 벡터로 임베딩하고 학습하지 않음
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # 2. 1D Convolution 레이어들 (다른 커널 크기를 사용)
        # filter_sizes 개수만큼의 Conv1d 레이어를 ModuleList로 생성
        # Conv1d는 (batch_size, in_channels, seq_len)을 입력으로 받음
        # 우리 임베딩은 (batch_size, seq_len, embed_dim)이므로, permute(0, 2, 1) 필요
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim,
                      out_channels=num_filters,
                      kernel_size=k) # n-gram 크기
            for k in filter_sizes
        ])

        # 3. 드롭아웃
        self.dropout = nn.Dropout(dropout_prob)

        # 4. FC 레이어 (분류기)
        # 각 필터에서 하나씩의 피처(max-pooling)가 나오므로,
        # 총 num_filters * len(filter_sizes) 개의 피처가 입력됨
        self.fc = nn.Linear(num_filters * len(filter_sizes), num_classes)

    def forward(self, input_ids):
        """
        모델의 순전파 로직

        Args:
            input_ids (torch.Tensor): (batch_size, seq_len)
                                     dataset.py에 의해 seq_len은 max_length-1이 됨

        Returns:
            torch.Tensor: (batch_size, num_classes)
                          각 클래스에 대한 logits
        """

        # 1. 임베딩
        # input_ids: (batch_size, seq_len)
        # embedded: (batch_size, seq_len, embed_dim)
        embedded = self.embedding(input_ids)

        # 2. Conv1d 입력을 위해 차원 변경
        # embedded: (batch_size, embed_dim, seq_len)
        embedded = embedded.permute(0, 2, 1)

        # 3. 컨볼루션 + ReLU
        # conved: (batch_size, num_filters, new_seq_len)
        conved = [F.relu(conv(embedded)) for conv in self.convs]

        # 4. Max pooling
        # F.max_pool1d(conv, conv.shape[2])는 (batch_size, num_filters, 1)을 반환
        # .squeeze(2)를 통해 (batch_size, num_filters)로 만듦
        # pooled: [ (batch_size, num_filters), (batch_size, num_filters), ... ]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        # 5. 피처 결합 (Concatenate)
        # catted: (batch_size, num_filters * len(filter_sizes))
        catted = torch.cat(pooled, dim=1)

        # 6. 드롭아웃
        dropped = self.dropout(catted)

        # 7. 완전 연결 레이어 (분류)
        # logits: (batch_size, num_classes)
        logits = self.fc(dropped)

        return logits

In [ ]:
# 1. 헬퍼 함수 정의
def count_parameters(model):
    """학습 가능한 파라미터 수 계산"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def epoch_time(start_time, end_time):
    """에폭 소요 시간 계산"""
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def calculate_accuracy(preds, y_true):
    """
    Accuracy 계산 함수
    logits(preds)를 받아서 argmax로 예측 클래스를 추출
    """
    y_pred = preds.argmax(dim=1) # (batch_size, num_classes) -> (batch_size)
    correct = (y_pred == y_true).float() # True/False를 1.0/0.0으로
    acc = correct.sum() / len(correct)
    return acc.item() # Python float 값으로 반환

In [ ]:
# 2. 훈련 함수 정의
def train(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        # 1. 배치 데이터를 device로 이동
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # 2. 그래디언트 초기화
        optimizer.zero_grad()

        # 3. 순전파
        # input_ids: (batch_size, seq_len)
        # predictions (logits): (batch_size, num_classes)
        predictions = model(input_ids)

        # 4. 손실 계산
        loss = criterion(predictions, labels)

        # 5. Accuracy 계산
        acc = calculate_accuracy(predictions, labels)

        # 6. 역전파
        loss.backward()

        # 7. 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # 8. 가중치 업데이트
        optimizer.step()

        # 9. 누적
        epoch_loss += loss.item()
        epoch_acc += acc

    # 평균 손실과 평균 acc 반환
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# 3. 평가 함수 정의
def evaluate(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            predictions = model(input_ids)
            loss = criterion(predictions, labels)

            # Accuracy 계산
            acc = calculate_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# 4. 테스트 로더용 예측 함수
def predict_test(model, iterator, device):
    """
    레이블이 없는 test_loader에 대해 예측을 수행하고
    (문장 ID 대신) 인덱스 순서대로 예측 클래스를 반환합니다.
    (submission.csv 생성을 위함)
    """
    model.eval()
    predictions_list = []

    with torch.no_grad():
        for batch in iterator:
            # test_loader는 'labels'가 없음
            input_ids = batch['input_ids'].to(device)

            # 모델 예측 (logits)
            predictions = model(input_ids)

            # 가장 확률이 높은 클래스 ID (0~4)
            y_pred = predictions.argmax(dim=1)

            predictions_list.extend(y_pred.cpu().numpy())

    return predictions_list

# 데이터 로드

In [ ]:
# ==================================================================
# 메인 실행 로직
# ==================================================================

# 0. GPU 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 1. 데이터 로더 준비
print("\nLoading data...")
TRAIN_PATH = './Data/aiffel-dl-thon-dktc-online-15/train.csv'
TEST_PATH = './Data/aiffel-dl-thon-dktc-online-15/test.csv'
VOCAB_SIZE = 1300
MAX_LENGTH = 400
BATCH_SIZE = 64
VALID_RATIO = 0.1 # 훈련 데이터 중 10%를 검증용으로 사용

# 2. 데이터 로더 생성
print("\nLoading data...")
try:
    # create_dataloaders는 훈련/테스트 로더와 vocab을 반환
    _train_loader, test_loader, vocab = create_dataloaders(
        TRAIN_PATH, TEST_PATH,
        vocab_size=VOCAB_SIZE,
        max_length=MAX_LENGTH,
        batch_size=BATCH_SIZE
    )
    PAD_IDX = vocab.PAD_ID # collate_fn에 사용

except FileNotFoundError:
    print("="*50)
    print("ERROR: 데이터 파일(train.csv/test.csv)을 찾을 수 없습니다.")
    print("TRAIN_PATH와 TEST_PATH를 실제 파일 경로로 수정해주세요.")
    print("="*50)
    exit()

# 3. 훈련/검증 데이터 분리
print("Splitting train data into train/validation sets...")

# DataLoader에서 원본 Dataset 객체 추출
train_dataset = _train_loader.dataset

# 데이터셋 크기 계산
total_size = len(train_dataset)
valid_size = int(total_size * VALID_RATIO)
train_size = total_size - valid_size

# torch.utils.data.random_split으로 데이터셋 분리
train_dataset, valid_dataset = torch.utils.data.random_split(
    train_dataset, [train_size, valid_size]
)

print(f"Total train samples: {total_size}")
print(f"  -> New Train set: {len(train_dataset)} samples")
print(f"  -> Validation set: {len(valid_dataset)} samples")

# 4. 분리된 데이터셋으로 DataLoader 재생성
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True, # 훈련 데이터는 셔플
    collate_fn=lambda batch: collate_fn(batch, pad_idx=PAD_IDX)
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # 검증 데이터는 셔플 안 함
    collate_fn=lambda batch: collate_fn(batch, pad_idx=PAD_IDX)
)

print("\nData loading and splitting complete.")

Using device: cuda

Loading data...

Loading data...
데이터 로드 및 전처리 중...
Train 데이터: 4950 개의 conversation
Test 데이터: 500 개의 conversation

샘플 데이터:
Conversation: 지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게. 정말 잘못했습니다. 너가 선택해. 너가 죽을래 네 가족을 죽여줄까. 죄송합니다. 정말 잘못했습니다. 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야. 선택 못하겠습니다. 한번만 도와주세요. 그냥 다 죽여버려야겠군. 이의 없지? 제발 도와주세요.
Label: 0

Conversation: 길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만 더 얘기한다. 장난전화 걸지 마시죠. 9시 40분 마트에 폭발물이 터지면 다 죽는거야. 장난전화는 업무방해죄에 해당됩니다. 판단은 너에게 달려있다. 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지. 선생님 진정하세요. 난 이야기했어. 경고했다는 말이야.
Label: 0

Conversation: 너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어. 그만해. 니들 놀리는거 재미없어. 지영아 너가 키 160이지? 그럼 재는 160도 안돼는거네? 너 군대도 안가고 좋겠다. 니들이 나 작은데 보태준거 있냐? 난쟁이들도 장가가고하던데. 너도 희망을 가져봐 더이상 하지마라. 그 키크는 수술도 있대잖아? 니네 엄마는 그거 안해주디? 나람 해줬어. 저 키로 어찌살아. 제발 그만 괴롭히라고!
Label: 3

SentencePiece 모델 학습 중...

모델 저장됨: ./configs/sentences.model
Vocab 크기: 1300

Train DataLoader 준비 완료: 총 4950개 conversations
Test DataLoader 준비 완료: 총 5

# 모델 학습

In [ ]:
# 5. 모델 하이퍼파라미터 및 초기화
print("\nInitializing 1D CNN model...")
INPUT_DIM = len(vocab)
EMBED_DIM = 256
NUM_CLASSES = 5
N_FILTERS = 128
FILTER_SIZES = [2, 3, 4, 5]
DROPOUT_PROB = 0.5

model = CNNClassifier(
    vocab_size=INPUT_DIM,
    embed_dim=EMBED_DIM,
    num_classes=NUM_CLASSES,
    num_filters=N_FILTERS,
    filter_sizes=FILTER_SIZES,
    dropout_prob=DROPOUT_PROB
).to(device)

print(f'The model has {count_parameters(model):,} trainable parameters.')

# 6. 옵티마이저 및 손실 함수 정의
LEARNING_RATE = 0.00005
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss().to(device)

# 7. 학습 설정
N_EPOCHS = 500
PATIENCE = 10
best_valid_loss = float('inf')
best_valid_acc = 0.0
patience_counter = 0
model_save_path = 'best_model_cnn.pt'


Initializing 1D CNN model...
The model has 794,629 trainable parameters.


In [ ]:
print(f"\n{'='*60}")
print(f"--- 1D CNN Model Training starts ---")
print(f"{'='*60}\n")

# 8. 학습 루프
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion, device)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    # 조기 종료 로직
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), model_save_path)
        patience_counter = 0
        print(f'\t>> Validation loss improved ({best_valid_loss:.3f}). Saving model...')
    else:
        patience_counter += 1
        print(f'\t>> Validation loss did not improve. Counter: {patience_counter}/{PATIENCE}')

        if patience_counter >= PATIENCE:
            print(f'--- Early stopping triggered after {epoch+1} epochs ---')
            break

print(f"\n{'='*60}")
print(f"--- Training Finished ---")
print(f"Best Model saved to: {model_save_path}")
print(f"  -> Best Validation Loss: {best_valid_loss:.3f}")
print(f"  -> Best Validation Acc at Best Loss: {best_valid_acc*100:.2f}%")
print(f"{'='*60}\n")


--- 1D CNN Model Training starts ---

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 1.799 | Train Acc: 24.20%
	 Val. Loss: 1.363 |  Val. Acc: 61.41%
	>> Validation loss improved (1.363). Saving model...
Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 1.502 | Train Acc: 38.48%
	 Val. Loss: 1.151 |  Val. Acc: 69.07%
	>> Validation loss improved (1.151). Saving model...
Epoch: 03 | Epoch Time: 0m 2s
	Train Loss: 1.316 | Train Acc: 46.24%
	 Val. Loss: 0.999 |  Val. Acc: 71.04%
	>> Validation loss improved (0.999). Saving model...
Epoch: 04 | Epoch Time: 0m 2s
	Train Loss: 1.162 | Train Acc: 54.18%
	 Val. Loss: 0.901 |  Val. Acc: 75.93%
	>> Validation loss improved (0.901). Saving model...
Epoch: 05 | Epoch Time: 0m 2s
	Train Loss: 1.046 | Train Acc: 59.08%
	 Val. Loss: 0.809 |  Val. Acc: 77.03%
	>> Validation loss improved (0.809). Saving model...
Epoch: 06 | Epoch Time: 0m 2s
	Train Loss: 0.971 | Train Acc: 62.99%
	 Val. Loss: 0.740 |  Val. Acc: 79.39%
	>> Validation loss improved (0.740). Savin

In [ ]:
print(f"\n{'='*60}")
print(f"--- Checking Model Predictions (1 Batch from Validation Set) ---")
print(f"{'='*60}\n")

idx_to_class = {
    0: '협박 대화', 1: '갈취 대화', 2: '직장 내 괴롭힘 대화',
    3: '기타 괴롭힘 대화', 4: '일반 대화'
}

# 1. 저장된 Best 모델 로드
try:
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)
    model.eval()
except FileNotFoundError:
    print(f"ERROR: 저장된 모델({model_save_path})을 찾을 수 없습니다.")
    exit()

# 2. 검증 데이터 1배치 가져오기
with torch.no_grad():
    # iter()로 DataLoader를 반복 가능한 객체로 만들고 next()로 1배치 추출
    try:
        batch = next(iter(valid_loader))
    except StopIteration:
        print("ERROR: valid_loader가 비어있습니다.")
        exit()

    input_ids = batch['input_ids'].to(device)
    labels = batch['labels'].to(device)

    # 3. 모델 예측 수행
    predictions = model(input_ids)
    y_pred = predictions.argmax(dim=1) # 예측 클래스 ID (0~4)

    # 4. 결과 비교 출력
    print(f"Total {len(labels)} samples in this batch.\n")

    for i in range(len(labels)):
        # 1) input_ids (텐서) -> list -> vocab으로 디코딩
        # <pad> 토큰(ID: 0)은 디코딩 시 제외
        token_ids = input_ids[i].cpu().tolist()

        # 0번(PAD_ID) 토큰을 제외하고 실제 텍스트로 디코딩
        # vocab.decode()는 dataset.py의 SentencePieceVocab 객체에 정의되어 있음
        # (BOS/EOS/CLS 등 특수 토큰은 vocab.decode()가 알아서 제외함)
        text = vocab.decode([tid for tid in token_ids if tid != PAD_IDX])

        pred_class_id = y_pred[i].item()
        true_class_id = labels[i].item()

        # 2) 예측 클래스와 실제 클래스 이름 가져오기
        pred_class_name = idx_to_class[pred_class_id]
        true_class_name = idx_to_class[true_class_id]

        # 3) 결과 출력
        is_correct = "✅ (Correct)" if pred_class_id == true_class_id else "❌ (WRONG)"

        print(f"--- Sample {i+1} / {is_correct} ---")
        print(f"  [Original Text]: {text}")
        print(f"  [Model Predict]: {pred_class_name} (ID: {pred_class_id})")
        print(f"  [Actual Label]:  {true_class_name} (ID: {true_class_id})")
        print("-" * 30)


--- Checking Model Predictions (1 Batch from Validation Set) ---

Total 64 samples in this batch.

--- Sample 1 / ✅ (Correct) ---
  [Original Text]: 야 왕따! .으.응.오.왜. 너 어제 공개적으로 차였다며? .어.떠.게 알.아.? 대놓고 고백한는데 그걸 모르는게 병신아님? 그렇.긴.한.데.못본 사람도있으니까 소문내지말아주라 제발. 왜?그럼 성의표시좀해봐! 알았어 내가 매점 다녀올께. 그래 5초준다 .아.아.알았어.
  [Model Predict]: 기타 괴롭힘 대화 (ID: 3)
  [Actual Label]:  기타 괴롭힘 대화 (ID: 3)
------------------------------
--- Sample 2 / ✅ (Correct) ---
  [Original Text]: 이 축구공 니거야? 네 제거예요 이거 형이 좀만 가지고 놀다 줄게 안돼요 저번에도 가지고 논다고 해놓고 잃어 버렸잖아요 너 형이 누군지 몰라? 학교 재미있게 다니게 해줄까? 아 형 진짜 안돼요 이거 삼촌한테 선물받은거라서 진짜 형이 화가 좀 나려고 하네 너 1학년 몇반이랬지? 아 죄송해요 형 근데 진짜 진짜 잃어버리면 안돼요 알았으니까 형 공가지고 놀게 안보이는데로 좀 가 네.
  [Model Predict]: 갈취 대화 (ID: 1)
  [Actual Label]:  갈취 대화 (ID: 1)
------------------------------
--- Sample 3 / ✅ (Correct) ---
  [Original Text]: 이마가 점점 넓어지는 것 같네 안 그래도 고민이다 야 넌 머리까지 없으면 너무 우울하다 안 그래도 스트레스다 돈도 없고 못생긴게 머리도 없으면 어쩌냐 그만해라 돈이라도 있으면 머리라도 심지 그만하라고 왜 나한테 그러냐 탈모는 유전인데 하
  [Model Predict]: 기타 괴롭힘 대화 (ID: 3)
  [Actual Labe

In [ ]:
# 9. 테스트 데이터 예측 및 제출 파일 생성
print(f"\n--- Loading best CNN model for test prediction ---")
try:
    model.load_state_dict(torch.load("/content/best_model_cnn_10.pt"))

    # test_loader로 예측 수행
    test_predictions = predict_test(model, test_loader, device)

    print("Prediction complete. Creating submission file...")

    import pandas as pd
    test_df = pd.read_csv(TEST_PATH)

    if len(test_df) == len(test_predictions):
        submission_df = pd.DataFrame({
            'idx': test_df['idx'],
            'class': test_predictions # <-- 숫자 ID 리스트를 그대로 사용
        })
        submission_df.to_csv('submission.csv', index=False)
        print("submission.csv file created successfully (with numeric IDs).")
    else:
        print(f"ERROR: Mismatch in length. Test DF: {len(test_df)}, Predictions: {len(test_predictions)}")
        print("Please check preprocessing logic if it removes test samples.")

except FileNotFoundError:
    print(f"ERROR: 저장된 모델({model_save_path})을 찾을 수 없습니다.")
except Exception as e:
    print(f"An error occurred during test prediction: {e}")


--- Loading best CNN model for test prediction ---
Prediction complete. Creating submission file...
submission.csv file created successfully (with numeric IDs).


In [ ]:
sub = pd.read_csv("submission.csv")
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   idx     500 non-null    object
 1   class   500 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


# 학습 기록
1차 시도:  
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.286
  -> Best Validation F1 (Macro) at Best Loss: 0.889
============================================================
```
<br>

2차 시도:
- L2 정칙화 추가  
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.333
  -> Best Validation F1 (Macro) at Best Loss: 0.888
============================================================
```
loss 크게 증가함. -> 다시 빼자  
<br>

3차 시도:  
- 정칙화 다시 제거
- N_FILTERS = 64 (128 -> 64)
- EMBED_DIM = 128 (256 -> 128)  
모델 크기 줄임
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.313
  -> Best Validation F1 (Macro) at Best Loss: 0.886
============================================================
```
<br>

4차 시도:  
- vocab_size = 1300 (1500 -> 1300)
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.294
  -> Best Validation F1 (Macro) at Best Loss: 0.887
============================================================
```
<br>

5차 시도:
- N_FILTERS = 128 (원상복구)
- EMBED_DIM = 256 (원상복구)
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.300
  -> Best Validation F1 (Macro) at Best Loss: 0.890
============================================================
```
다 비슷비슷한데,,, 지금까진 모두 얼리스탑했으니 학습률 조정하고 에포크 늘려보자.  
<br>

6차 시도:  
- 평가 방법 변경 (f1 -> Acc)
- lr = 0.0001 (0.001 -> 0.0001)
- epochs = 500 (30 -> 500)
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.239
  -> Best Validation Acc at Best Loss: 91.21%
============================================================
```
45 epoch에서 early stop  
<br>

7차 시도:  
- FILTER_SIZES = [2, 3, 4, 5]   
    ([3, 4, 5] -> [2, 3, 4, 5])
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.233
  -> Best Validation Acc at Best Loss: 91.80%
============================================================
```
57 epoch에서 early stop  
<br>

8차 시도:  
- dropout = 0.3 (0.5 -> 0.3)
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.237
  -> Best Validation Acc at Best Loss: 91.02%
============================================================
```
늘려서 다시 해보자  
<br>

9차 시도:  
- dropout = 0.7 (0.3 -> 0.7)
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.241
  -> Best Validation Acc at Best Loss: 91.14%
============================================================
```
큰 차이 없어 보임.  
<br>

10차 시도:  
- lr 줄였으니 L2 정칙화 다시 시도  
- 라고 하려고 했으나 지금까지 정칙화 계속 적용하고 있었음;;  
- 이번엔 없애서 해보자  
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.222
  -> Best Validation Acc at Best Loss: 92.90%
============================================================
```
ridge 없애니 확실히 올랐음.  
<br>

11차 시도:  
- 다시 모델 크기 줄여보기
- N_FILTERS = 64 (128 -> 64)
- EMBED_DIM = 128 (256 -> 128)  
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.276
  -> Best Validation Acc at Best Loss: 90.29%
============================================================
```
  
<br>

12차 시도:  
- N_FILTERS = 256 (64 -> 256)
- EMBED_DIM = 512 (128 -> 512)  
-> 파라미터 250만개
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.230
  -> Best Validation Acc at Best Loss: 92.58%
============================================================
```
좋긴 한데,, 학습률 더 낮춰보자  
<br>

13차 시도:  
- lr = 1e-5 (0.0001 -> 1e-5)
```
============================================================
--- Training Finished ---
Best Model saved to: best_model_cnn.pt
  -> Best Validation Loss: 0.290
  -> Best Validation Acc at Best Loss: 90.15%
============================================================
```
의미 없는 듯 하다.  
<br>


1D CNN 선택 이유:  
- 협박, 갈취 등 자극적인 대화는 "죽어", "내놔" 같은 짧은 키워드로 구분될 가능성이 높다고 생각했다.  
- 따라서 n-gram 방식을 사용하며 국소적인 패턴을 감지하는 데 좋은 성능을 보이는 1d CNN을 선택했다.  
- 또한, 구조가 다른 모델에 비해 단순해서 학습에 걸리는 시간이 상대적으로 짧다.
